In [ ]:
from ortools.sat.python import cp_model
import pandas as pd
import json
from tabulate import tabulate
import time
import sys

# --------------------
# Solution Callback Class
# --------------------
class TimetableSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, divisions, days, slots, all_classes, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._variables = variables
        self._divisions = divisions
        self._days = days
        self._slots = slots
        self._all_classes = all_classes
        self._solution_count = 0
        self._solution_limit = limit
        self._start_time = time.time()

    def on_solution_callback(self):
        current_time = time.time()
        print(f'Solution {self._solution_count} found in {current_time - self._start_time:.2f} seconds')
        self._solution_count += 1
        if self._solution_count >= self._solution_limit:
            print(f'Stopping search after finding {self._solution_limit} solutions.')
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

def load_data_from_json():
    """Loads all necessary data from external JSON files."""
    try:
        with open('config.json', 'r') as f:
            config = json.load(f)
        with open('frequencies.json', 'r') as f:
            class_frequencies = json.load(f)
        with open('teachers.json', 'r') as f:
            teacher_assignments = json.load(f)
        with open('rooms.json', 'r') as f:
            room_assignments = json.load(f)
        
        print("Successfully loaded all JSON configuration files.")
        return config, class_frequencies, teacher_assignments, room_assignments
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure config.json, frequencies.json, teachers.json, and rooms.json are in the same directory.")
        sys.exit(1)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}. Please check the format of your JSON files.")
        sys.exit(1)


def main():
    # --------------------
    # Parameters from JSON
    # --------------------
    config, class_frequencies, teacher_assignments, room_assignments = load_data_from_json()

    divisions = config['divisions']
    lectures = config['lectures']
    labs = config['labs']
    days = config['days']
    
    # --- Dynamic Time Slot Calculation ---
    start_hour = config['start_hour']
    end_hour = config['end_hour']
    lunch_start_hour = config.get('lunch_start_hour') # .get makes it optional

    if start_hour >= end_hour:
        print("Error: start_hour must be less than end_hour in config.json")
        sys.exit(1)

    total_hours = end_hour - start_hour
    slots = list(range(total_hours))
    
    # Calculate lunch slot index dynamically
    LUNCH_SLOT = None
    if lunch_start_hour is not None and start_hour <= lunch_start_hour < end_hour:
        LUNCH_SLOT = lunch_start_hour - start_hour
        print(f"Lunch break is scheduled for slot index {LUNCH_SLOT} ({lunch_start_hour}:00 - {lunch_start_hour+1}:00).")
    else:
        print("No lunch break is scheduled within the specified college hours.")


    all_classes = lectures + labs
    num_classes = len(all_classes)
    class_to_idx = {cls: i for i, cls in enumerate(all_classes)}
    idx_to_class = {i: cls for cls, i in class_to_idx.items()}

    # --- Reconstruct derived data ---
    class_to_teacher = {cls: teacher for teacher, classes in teacher_assignments.items() for cls in classes}
    class_to_room = {cls: room for room, classes in room_assignments.items() for cls in classes}
    all_teachers = list(teacher_assignments.keys())
    all_rooms = list(room_assignments.keys())


    # --------------------
    # Model & Variables
    # --------------------
    model = cp_model.CpModel()
    x = {}
    for d in divisions:
        x[d] = {}
        for day in days:
            x[d][day] = {}
            for s in slots:
                x[d][day][s] = model.NewIntVar(-1, num_classes - 1, f'x_{d}_{day}_{s}')

    # Add lunch break constraint only if it's defined
    if LUNCH_SLOT is not None:
        for d in divisions:
            for day in days:
                model.Add(x[d][day][LUNCH_SLOT] == -1)

    # --------------------
    # CONSTRAINTS (per division)
    # --------------------
    for d in divisions:
        # --- Lab Constraints (Frequency and Daily Limit) ---
        for lab in labs:
            lab_val = class_to_idx[lab]
            possible_starts_week = []
            for day in days:
                possible_starts_day = []
                # Iterate through possible start slots for a 2-hour lab
                for s in range(len(slots) - 1):
                    # Prevent labs from starting in or crossing the lunch break
                    if LUNCH_SLOT is not None and (s == LUNCH_SLOT or s + 1 == LUNCH_SLOT):
                        continue
                    
                    b = model.NewBoolVar(f'b_{d}_{lab}_{day}_{s}')
                    possible_starts_day.append(b)
                    
                    # If this bool is true, then this slot and the next are the lab
                    model.Add(x[d][day][s] == lab_val).OnlyEnforceIf(b)
                    model.Add(x[d][day][s+1] == lab_val).OnlyEnforceIf(b)

                # Constraint: No more than one lab session for the same lab on the same day
                model.Add(sum(possible_starts_day) <= 1)
                possible_starts_week.extend(possible_starts_day)

            # Constraint: Each lab must be scheduled for its specified frequency per week
            model.Add(sum(possible_starts_week) == class_frequencies[lab])

        # --- Lecture Constraints (Frequency and Daily Limit) ---
        for lec in lectures:
            lec_val = class_to_idx[lec]
            bool_vars_week = []
            for day in days:
                bool_vars_day = []
                for s in slots:
                    b = model.NewBoolVar(f'b_{d}_{lec}_{day}_{s}')
                    model.Add(x[d][day][s] == lec_val).OnlyEnforceIf(b)
                    bool_vars_day.append(b)
                
                # Constraint: No more than one lecture of the same type per day
                model.Add(sum(bool_vars_day) <= 1)
                bool_vars_week.extend(bool_vars_day)

            # Constraint: Each lecture must be scheduled for its specified frequency per week
            model.Add(sum(bool_vars_week) == class_frequencies[lec])


    # --------------------
    # OBJECTIVE FUNCTION (Minimize Gaps)
    # --------------------
    objective_terms = []
    for d in divisions:
        for day in days:
            for s in range(len(slots) - 1):
                if LUNCH_SLOT is not None and (s == LUNCH_SLOT - 1 or s == LUNCH_SLOT):
                    continue

                current_slot_var = x[d][day][s]
                next_slot_var = x[d][day][s + 1]

                class_then_free = model.NewBoolVar(f'class_then_free_{d}_{day}_{s}')
                model.Add(current_slot_var != -1).OnlyEnforceIf(class_then_free)
                model.Add(next_slot_var == -1).OnlyEnforceIf(class_then_free)

                free_then_class = model.NewBoolVar(f'free_then_class_{d}_{day}_{s}')
                model.Add(current_slot_var == -1).OnlyEnforceIf(free_then_class)
                model.Add(next_slot_var != -1).OnlyEnforceIf(free_then_class)
                
                objective_terms.append(class_then_free)
                objective_terms.append(free_then_class)
    model.Minimize(sum(objective_terms))


    # --------------------
    # GLOBAL CONSTRAINTS (across all divisions)
    # --------------------
    for day in days:
        for s in slots:
            all_slots_vars = [x[d][day][s] for d in divisions]
            
            # --- Teacher Clash Constraint ---
            for teacher in all_teachers:
                classes_taught = [class_to_idx[c] for c in teacher_assignments.get(teacher, [])]
                if not classes_taught: continue
                
                bool_vars = []
                for cls_idx in classes_taught:
                    for slot_var in all_slots_vars:
                        b = model.NewBoolVar('')
                        model.Add(slot_var == cls_idx).OnlyEnforceIf(b)
                        model.Add(slot_var != cls_idx).OnlyEnforceIf(b.Not())
                        bool_vars.append(b)
                model.Add(sum(bool_vars) <= 1)

            # --- Room Clash Constraint ---
            for room in all_rooms:
                classes_in_room = [class_to_idx[c] for c in room_assignments.get(room, [])]
                if not classes_in_room: continue

                bool_vars = []
                for cls_idx in classes_in_room:
                    for slot_var in all_slots_vars:
                        b = model.NewBoolVar('')
                        model.Add(slot_var == cls_idx).OnlyEnforceIf(b)
                        model.Add(slot_var != cls_idx).OnlyEnforceIf(b.Not())
                        bool_vars.append(b)
                model.Add(sum(bool_vars) <= 1)

    # --------------------
    # Solve
    # --------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 180.0
    solver.parameters.num_search_workers = 8 
    solver.parameters.log_search_progress = True
    
    print("Starting solver... this may take a few minutes.")
    status = solver.Solve(model)
    print("Solver finished.")

    # --------------------
    # Generate Timetable & JSON
    # --------------------
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        print(f"\nSolution found! Status: {solver.StatusName(status)}")
        timetable_json = {}
        for d in divisions:
            table_data = []
            json_data = {}
            time_headers = [f'{start_hour+i}:00-{start_hour+i+1}:00' for i in slots]
            
            for day in days:
                formatted_row, json_row = [], []
                for s in slots:
                    if s == LUNCH_SLOT:
                        json_cell = {"class": "LUNCH", "teacher": "", "room": ""}
                        formatted_cell_str = "LUNCH\n-\n-"
                    else:
                        val = solver.Value(x[d][day][s])
                        class_name = idx_to_class.get(val, "Free")
                        
                        # Determine if it's a continuation of a lab
                        is_continuation = False
                        if s > 0 and val != -1 and class_name in labs:
                            if solver.Value(x[d][day][s-1]) == val:
                                is_continuation = True

                        # Create the JSON cell for the output file
                        if class_name != "Free":
                            json_cell = {"class": class_name, "teacher": class_to_teacher.get(class_name, ""), "room": class_to_room.get(class_name, "")}
                        else:
                            json_cell = {"class": "Free", "teacher": "", "room": ""}

                        # Format the string for the visual table
                        if is_continuation:
                            formatted_cell_str = "(continued)\n\n"
                        elif class_name == "Free":
                            formatted_cell_str = "-\n-\n-"
                        else:
                            formatted_cell_str = f"{json_cell['class']}\n{json_cell['teacher']}\n{json_cell['room']}"
                    
                    json_row.append(json_cell)
                    formatted_row.append(formatted_cell_str)
                    
                table_data.append([day] + formatted_row)
                json_data[day] = json_row
            
            df = pd.DataFrame(table_data, columns=['Day'] + time_headers)
            print(f'\n=============== Timetable for {d} ===============')
            print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))
            timetable_json[d] = json_data
            
        with open("timetable_full.json", "w") as f:
            json.dump(timetable_json, f, indent=4)
        print("\nTimetable successfully exported to timetable_full.json")
    else:
        print("\nNo solution found. The problem may be infeasible or requires more time.")
        print(f"Solver status: {solver.StatusName(status)}")

if __name__ == "__main__":
    main()



Successfully loaded all JSON configuration files.
Lunch break is scheduled for slot index 3 (12:00 - 13:00).
Starting solver... this may take a few minutes.
Solver finished.

Solution found! Status: OPTIMAL

=============== Timetable for DivA ===============
+-------+--------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
| Day   | 9:00-10:00   | 10:00-11:00   | 11:00-12:00   | 12:00-13:00   | 13:00-14:00   | 14:00-15:00   | 15:00-16:00   | 16:00-17:00   |
+=======+==============+===============+===============+===============+===============+===============+===============+===============+
| Mon   | UHV          | -             | -             | LUNCH         | DSA           | Finance       | MA            | -             |
|       | Prof4        | -             | -             | -             | Prof1         | Prof6         | Prof2         | -             |
|       | Room104      | -             | -             |